In [43]:
import os
%pwd
new_directory = r"C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS"

# Change the current working directory
os.chdir(new_directory)

In [44]:
%pwd

'C:\\Users\\aditi\\OneDrive\\Desktop\\Chest-Cancer-ML-OPS'

In [45]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class TrainModelConfig:
    root_dir : Path
    trained_model_path :Path
    updated_base_model_path : Path
    training_data : Path
    epochs_params : int
    batch_size_params : int
    is_augmentation_params : bool
    image_size_params : list
    

In [46]:
from chest_cancer_classifier.constants import *
from chest_cancer_classifier.utils.common import read_yaml, create_dir
import tensorflow as tf

In [47]:
class ConfigManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        param_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(param_filepath)
        
        create_dir([self.config.artifacts_root])
        
    def train_model_config(self) -> TrainModelConfig:
        training = self.config.train_model
        prepare_base_model = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir,"Data\\test")
        create_dir([Path(training.root_dir)])
        
        training_config = TrainModelConfig(
            root_dir= Path(training.root_dir),
            trained_model_path = Path(training.trained_model_path),
            updated_base_model_path = Path(prepare_base_model.updated_base_model_path),
            training_data=Path(training_data),
            epochs_params=params.EPOCHS,
            batch_size_params=params.BATCH_SIZE,
            is_augmentation_params=params.AUGMENTATION,
            image_size_params = params.IMAGE_SIZE,
          
        )
        return training_config
        

In [48]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [49]:
import tensorflow as tf
from pathlib import Path

class Training:
    def __init__(self, config: TrainModelConfig):
        self.config = config
        
    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            self.config.updated_base_model_path
        )
        
    def train_valid_gen(self):
        
        datagen_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )
        
        dataflow_kwargs = dict(
            target_size=self.config.image_size_params[:-1],
            batch_size=self.config.batch_size_params,
            interpolation="bilinear"
        )
        
        valid_data_gen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagen_kwargs
        )
        
        self.valid_gen = valid_data_gen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )
        
        if self.config.is_augmentation_params:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            train_datagen = valid_data_gen
            
        self.train_gen = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
        
    def train(self):
        self.steps_per_epoch = self.train_gen.samples // self.train_gen.batch_size
        self.validation_steps = self.valid_gen.samples // self.valid_gen.batch_size

        # Callbacks for learning rate reduction and early stopping
        lr_reduce = tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            verbose=1
        )

        early_stop = tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            verbose=1,
            restore_best_weights=True
        )

        self.model.fit(
            self.train_gen,
            epochs=self.config.epochs_params,
            steps_per_epoch=self.steps_per_epoch,
            validation_steps=self.validation_steps,
            validation_data=self.valid_gen,
            callbacks=[lr_reduce, early_stop]
        )
        
        self.save_model(
            path=self.config.trained_model_path,
            model=self.model
        )


In [50]:
try:
    config= ConfigManager()
    training_config = config.train_model_config()
    training =Training(config = training_config)
    training.get_base_model()
    training.train_valid_gen()
    training.train()
except Exception as e:
    raise e

path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml
[2024-08-31 22:26:36,647 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\config\config.yaml successfully loaded]
path C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml
[2024-08-31 22:26:36,656 INFO : common : YAML file C:\Users\aditi\OneDrive\Desktop\Chest-Cancer-ML-OPS\params.yaml successfully loaded]
[2024-08-31 22:26:36,659 INFO : common : Created Directory at : artifacts]
[2024-08-31 22:26:36,662 INFO : common : Created Directory at : artifacts\training]
Found 178 images belonging to 4 classes.
Found 722 images belonging to 4 classes.
45/45 [==============================] - 532s 12s/step - loss: 22.0031 - accuracy: 0.2960 - val_loss: 26.3293 - val_accuracy: 0.1818 - lr: 0.0100
